In [ ]:
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


In [ ]:
from datasets import load_dataset
dataset = load_dataset('glue', 'mrpc')
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [ ]:
!pip install -q transformers

In [ ]:
!pip install -q tqdm

In [ ]:
from datasets import load_dataset
from transformers import pipeline
from tqdm.auto import tqdm  # Import tqdm for progress bar

# Load the MRPC dataset
dataset = load_dataset('glue', 'mrpc')

# Load a pre-trained sentiment analysis pipeline
sentiment_pipeline = pipeline("sentiment-analysis", model="finiteautomata/bertweet-base-sentiment-analysis")

def add_sentiment_to_dataset(dataset):
    for split in dataset:
        sentiments1 = []
        sentiments2 = []

        # Using tqdm to show progress bar
        for item in tqdm(dataset[split], desc=f"Processing {split} split"):
            # Predict sentiment for sentence1 and sentence2
            sentiment1 = sentiment_pipeline(item['sentence1'])[0]
            sentiment2 = sentiment_pipeline(item['sentence2'])[0]

            sentiments1.append(sentiment1)
            sentiments2.append(sentiment2)

        # Add the sentiment predictions to the dataset
        dataset[split] = dataset[split].add_column('sentiment1', sentiments1)
        dataset[split] = dataset[split].add_column('sentiment2', sentiments2)

    return dataset

# Add sentiment to the dataset
dataset_with_sentiment = add_sentiment_to_dataset(dataset)

# Now dataset_with_sentiment has two new columns: 'sentiment1' and 'sentiment2'


config.json:   0%|          | 0.00/949 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/338 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/843k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


Processing train split:   0%|          | 0/3668 [00:00<?, ?it/s]

Processing validation split:   0%|          | 0/408 [00:00<?, ?it/s]

Processing test split:   0%|          | 0/1725 [00:00<?, ?it/s]

In [ ]:
dataset_with_sentiment

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'sentiment1', 'sentiment2'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'sentiment1', 'sentiment2'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'sentiment1', 'sentiment2'],
        num_rows: 1725
    })
})

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/Colab Notebooks/5_Corpora/mrpc-sentiment-triplet-1056-source.csv'

Mounted at /content/drive


In [ ]:
import pandas as pd

# Assuming dataset_with_sentiment is your dataset with the added sentiment columns

# Convert each split to a DataFrame
train_df = pd.DataFrame(dataset_with_sentiment['train'])
validation_df = pd.DataFrame(dataset_with_sentiment['validation'])
test_df = pd.DataFrame(dataset_with_sentiment['test'])

# Combine the DataFrames
combined_df = pd.concat([train_df, validation_df, test_df], ignore_index=True)

In [ ]:
combined_df.head()

,sentence1,sentence2,label,idx,sentiment1,sentiment2
0,"Amrozi accused his brother , whom he called "" ...","Referring to him as only "" the witness "" , Amr...",1,0,"{'label': 'NEG', 'score': 0.8856388330459595}","{'label': 'NEG', 'score': 0.7828174233436584}"
1,Yucaipa owned Dominick 's before selling the c...,Yucaipa bought Dominick 's in 1995 for $ 693 m...,0,1,"{'label': 'NEU', 'score': 0.942793607711792}","{'label': 'NEU', 'score': 0.9274392127990723}"
2,They had published an advertisement on the Int...,"On June 10 , the ship 's owners had published ...",1,2,"{'label': 'NEU', 'score': 0.9626546502113342}","{'label': 'NEU', 'score': 0.7317103147506714}"
3,"Around 0335 GMT , Tab shares were up 19 cents ...","Tab shares jumped 20 cents , or 4.6 % , to set...",0,3,"{'label': 'NEU', 'score': 0.8141699433326721}","{'label': 'POS', 'score': 0.7458325028419495}"
4,"The stock rose $ 2.11 , or about 11 percent , ...",PG & E Corp. shares jumped $ 1.63 or 8 percent...,1,4,"{'label': 'NEU', 'score': 0.6198355555534363}","{'label': 'NEU', 'score': 0.7615814805030823}"


In [ ]:
import pandas as pd
import json

# Assuming combined_df is your combined DataFrame

# Check if the sentiment columns need to be parsed from JSON string to Python dict
# If they are already in dict format, you can skip the json.loads part
def parse_sentiment(sentiment):
    if isinstance(sentiment, str):
        return json.loads(sentiment)
    return sentiment

# Apply parsing to sentiment columns (if needed)
combined_df['sentiment1'] = combined_df['sentiment1'].apply(parse_sentiment)
combined_df['sentiment2'] = combined_df['sentiment2'].apply(parse_sentiment)

# Create new columns for sentiment label and score
combined_df['sentiment1_label'] = combined_df['sentiment1'].apply(lambda x: x['label'])
combined_df['sentiment1_score'] = combined_df['sentiment1'].apply(lambda x: x['score'])
combined_df['sentiment2_label'] = combined_df['sentiment2'].apply(lambda x: x['label'])
combined_df['sentiment2_score'] = combined_df['sentiment2'].apply(lambda x: x['score'])

# Now, you can drop the original sentiment1 and sentiment2 columns if they are no longer needed
combined_df = combined_df.drop(['sentiment1', 'sentiment2'], axis=1)

# You can now save this DataFrame or use it for further processing


In [ ]:
combined_df.head()

,sentence1,sentence2,label,idx,sentiment1_label,sentiment1_score,sentiment2_label,sentiment2_score
0,"Amrozi accused his brother , whom he called "" ...","Referring to him as only "" the witness "" , Amr...",1,0,NEG,0.885639,NEG,0.782817
1,Yucaipa owned Dominick 's before selling the c...,Yucaipa bought Dominick 's in 1995 for $ 693 m...,0,1,NEU,0.942794,NEU,0.927439
2,They had published an advertisement on the Int...,"On June 10 , the ship 's owners had published ...",1,2,NEU,0.962655,NEU,0.731710
3,"Around 0335 GMT , Tab shares were up 19 cents ...","Tab shares jumped 20 cents , or 4.6 % , to set...",0,3,NEU,0.814170,POS,0.745833
4,"The stock rose $ 2.11 , or about 11 percent , ...",PG & E Corp. shares jumped $ 1.63 or 8 percent...,1,4,NEU,0.619836,NEU,0.761581


In [ ]:
# Assuming combined_df is your DataFrame with the sentiment label columns

# Count values in sentiment1_label
sentiment1_label_counts = combined_df['sentiment1_label'].value_counts()

# Count values in sentiment2_label
sentiment2_label_counts = combined_df['sentiment2_label'].value_counts()

# Print the counts
print("Sentiment1 Label Counts:\n", sentiment1_label_counts)
print("\nSentiment2 Label Counts:\n", sentiment2_label_counts)


Sentiment1 Label Counts:
 NEU    3702
NEG    1479
POS     620
Name: sentiment1_label, dtype: int64

Sentiment2 Label Counts:
 NEU    3690
NEG    1481
POS     630
Name: sentiment2_label, dtype: int64


In [ ]:
# Assuming combined_df is your DataFrame

# Filter the DataFrame where sentiment1_label is not equal to sentiment2_label
filtered_df = combined_df[combined_df['sentiment1_label'] != combined_df['sentiment2_label']]

# Now filtered_df contains only the records where both sentiment labels are the same


In [ ]:
filtered_df

,sentence1,sentence2,label,idx,sentiment1_label,sentiment1_score,sentiment2_label,sentiment2_score
3,"Around 0335 GMT , Tab shares were up 19 cents ...","Tab shares jumped 20 cents , or 4.6 % , to set...",0,3,NEU,0.814170,POS,0.745833
6,"The Nasdaq had a weekly gain of 17.27 , or 1.2...",The tech-laced Nasdaq Composite .IXIC rallied ...,0,6,NEU,0.856691,POS,0.534851
8,"That compared with $ 35.18 million , or 24 cen...",Earnings were affected by a non-recurring $ 8 ...,0,8,NEU,0.973141,NEG,0.682705
9,"Shares of Genentech , a much larger company wi...",Shares of Xoma fell 16 percent in early trade ...,0,10,POS,0.834054,NEU,0.523223
14,"Gyorgy Heizler , head of the local disaster un...","The head of the local disaster unit , Gyorgy H...",0,15,NEU,0.968184,NEG,0.742662
...,...,...,...,...,...,...,...,...
5768,The stunning art robbery on Sunday was one of ...,Art historians said it was one of the signific...,1,1692,NEU,0.635170,NEG,0.944184
5770,Bush turned out a statement yesterday thanking...,Mr. Bush did not discuss this when he issued a...,0,1694,POS,0.800994,NEU,0.856950
5773,"Pataki praised Abraham 's decision , and LIPA ...",LIPA Chairman Richard Kessel said that meant t...,0,1697,POS,0.852039,NEU,0.709880
5795,"The deal , approved by both companies ' board ...",The acquisition has been approved by both comp...,1,1719,POS,0.564875,NEU,0.651503


In [ ]:
# Assuming combined_df is your DataFrame with the sentiment label columns

# Count values in sentiment1_label
sentiment1_label_counts = filtered_df['sentiment1_label'].value_counts()

# Count values in sentiment2_label
sentiment2_label_counts = filtered_df['sentiment2_label'].value_counts()

# Print the counts
print("Sentiment1 Label Counts:\n", sentiment1_label_counts)
print("\nSentiment2 Label Counts:\n", sentiment2_label_counts)

Sentiment1 Label Counts:
 NEU    523
NEG    327
POS    206
Name: sentiment1_label, dtype: int64

Sentiment2 Label Counts:
 NEU    511
NEG    329
POS    216
Name: sentiment2_label, dtype: int64


In [ ]:
# Save the combined DataFrame to a CSV file
filtered_df.to_csv(file_path, index=False)